In [ ]:
import kagglehub
import pandas as pd
import numpy as np
import seaborn as sns
import ast
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt

# Download latest version
path = kagglehub.dataset_download("tmdb/tmdb-movie-metadata")

print("Path to dataset files:", path)

In [ ]:
raw_csv_movies = pd.read_csv(path + "/tmdb_5000_movies.csv")
raw_csv_credits = pd.read_csv(path + "/tmdb_5000_credits.csv")

In [ ]:
raw_csv_movies

In [ ]:
raw_csv_credits

In [ ]:
df_movies = raw_csv_movies.copy()
df_credits = raw_csv_credits.copy()

In [ ]:
df_movies.info()

In [ ]:
df_credits.info()

In [ ]:
missing_revenue = df_movies['revenue'].isna().sum()
missing_budget = df_movies['budget'].isna().sum()
missing_runtime = df_movies['runtime'].isna().sum()

In [ ]:
missing_revenue, missing_budget, missing_runtime

In [ ]:
invalid_mask = (
    df_movies['revenue'].isna() | 
    (df_movies['revenue'] == 0) | 
    df_movies['budget'].isna() | 
    (df_movies['budget'] == 0)
)

In [ ]:
clean_df = df_movies[~invalid_mask].copy()
clean_df.reset_index(drop=True, inplace=True)

In [ ]:
clean_df

In [ ]:
valid_runtimes = clean_df.loc[(clean_df['runtime'].notna()) & (clean_df['runtime'] > 0), 'runtime']
runtime_median = valid_runtimes.median()

In [ ]:
runtime_median

In [ ]:
clean_df['runtime'] = np.where(
    (clean_df['runtime'].isna()) | (clean_df['runtime'] == 0),  # Condition
    runtime_median,                                  # Value if true
    clean_df['runtime']                                    # Value if false
)

In [ ]:
clean_df['runtime'] = clean_df['runtime'].astype(int)

In [ ]:
print("Original date format and examples:")
print(f"Data type: {clean_df['release_date'].dtype}")
print("Sample dates before conversion:")
print(clean_df['release_date'].sample(5).to_string(index=False))

In [ ]:
date_df = clean_df.copy()

In [ ]:
print("Original date format and examples:")
print(f"Data type: {date_df['release_date'].dtype}")
print("Sample dates before conversion:")
print(date_df['release_date'].sample(5).to_string(index=False))

In [ ]:
date_df['release_date'] = pd.to_datetime(
    date_df['release_date'],
    yearfirst=True,  # Specify if year is first in the format
)

In [ ]:
print("\nAfter conversion:")
print(f"New data type: {date_df['release_date'].dtype}")
print(f"Missing/invalid dates: {date_df['release_date'].isna().sum()}")
date_df

In [ ]:
date_df.info()

In [ ]:
movie_genres = date_df['genres']

In [ ]:
movie_genres

In [ ]:
# Extract genre names from the movie_genres column
movie_genre_names = []
for genres in movie_genres:
    genre_list = ast.literal_eval(genres)  # Convert string representation of list to actual list
    movie_genre_names.extend([genre['name'] for genre in genre_list])

# Remove duplicates and sort the list
movie_genre_names = sorted(set(movie_genre_names))
movie_genre_names

In [ ]:
dublicate_movies = date_df.copy()

In [ ]:
dublicate_movies

In [ ]:
duplicates_mask = dublicate_movies.duplicated(subset=['title'], keep=False)
duplicate_titles = dublicate_movies[duplicates_mask].sort_values('title')

In [ ]:
print(f"Found {len(duplicate_titles)} potential duplicates")
print(duplicate_titles[['title', 'release_date', 'id']].head(10))


In [ ]:
duplicate_years = duplicate_titles.groupby('title')['release_date'].nunique()

In [ ]:
print("Titles with multiple release years:")
print(duplicate_years[duplicate_years > 1])

In [ ]:
dublicate_movies['profit'] = dublicate_movies['revenue'] - dublicate_movies['budget']
dublicate_movies['ROI'] = (dublicate_movies['profit']/dublicate_movies['budget'].replace(np.inf, np.nan))

In [ ]:
dublicate_movies

In [ ]:
dublicate_movies['release_season'] = dublicate_movies['release_date'].dt.month.apply(
    lambda x: 'Winter' if x <= 2 else 
             'Spring' if x <= 5 else 
             'Summer' if x <= 8 else 
             'Fall'
)

In [ ]:
dublicate_movies

In [ ]:
df_preprosessed = dublicate_movies.copy()

In [ ]:
df_preprosessed

In [ ]:
# Extract genre names for each movie
df_preprosessed['genres_list'] = df_preprosessed['genres'].apply(lambda x: [genre['name'] for genre in ast.literal_eval(x)])

# Perform one-hot encoding
genres_one_hot = df_preprosessed['genres_list'].explode().str.get_dummies().groupby(level=0).sum()

# Add the one-hot encoded genres back to the original DataFrame
df_preprosessed = pd.concat([df_preprosessed, genres_one_hot], axis=1)

df_preprosessed.head()

In [ ]:
scaler = MinMaxScaler()
df_preprosessed[['budget_scaled', 'revenue_scaled']] = scaler.fit_transform(df_preprosessed[['budget', 'revenue']])

In [ ]:
df_preprosessed

In [ ]:
df_preprosessed.to_csv(path + "/tmdb_preprocessed.csv", sep=",", index=False)

In [ ]:
# Basic info
print(f"Total movies: {len(df_preprosessed)}")
print(f"Columns: {df_preprosessed.columns.tolist()}")

# Summary stats for numerical columns
print("\nSummary Statistics:")
print(df_preprosessed[['budget', 'revenue', 'runtime', 'vote_average']].describe())

In [ ]:
plt.figure(figsize=(10, 5))
plt.hist(df_preprosessed['budget'], bins=30, edgecolor='black')
plt.title("Distribution of Movie Budgets")
plt.xlabel("Budget (in $100 millions)")
plt.ylabel("Number of Movies")
plt.show()

In [ ]:
# Sum all genre columns (Action, Adventure, etc.)
genre_counts = df_preprosessed[movie_genre_names].sum().sort_values()

# Plot
plt.figure(figsize=(10, 6))
genre_counts.plot(kind='barh')
plt.title("Most Common Movie Genres")
plt.xlabel("Number of Movies")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(df_preprosessed['budget'], df_preprosessed['revenue'], alpha=0.5)
plt.title("Budget vs. Revenue")
plt.xlabel("Budget")
plt.ylabel("Revenue")
plt.show()

In [ ]:
plt.figure(figsize=(15, 6))
sns.boxplot(data=df_preprosessed, x='vote_average', y='profit', hue='release_season')
plt.xticks(rotation=90)
plt.title("Profit by Movie Rating and Release Season")
plt.show()

In [ ]:
plt.figure(figsize=(10, 6))
sns.lineplot(data=df_preprosessed, x='release_date', y='runtime')
plt.title("Average Runtime Over Time")
plt.show()

In [ ]:
# Movies with extreme ROI
weird_movies = df_preprosessed[(df_preprosessed['ROI'] > 10) | (df_preprosessed['ROI'] < -0.5)]
print("Movies with unusual ROI:")
print(weird_movies[['title', 'budget', 'revenue', 'ROI']])

In [ ]:
corr_matrix = df_preprosessed[['budget', 'revenue', 'runtime', 'vote_average', 'profit']].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.title("Correlation Matrix")
plt.show()

In [ ]:
season_profit = df_preprosessed.groupby('release_season')['profit'].mean().sort_values()

plt.figure(figsize=(10, 6))
season_profit.plot(kind='bar')
plt.title("Average Profit by Release Season")
plt.ylabel("Profit (in billions)")
plt.show()

In [ ]:
# Calculate average profit per genre
genre_profit = {}
for genre in movie_genre_names:
    genre_profit[genre] = df_preprosessed[df_preprosessed[genre] == 1]['profit'].mean()

# Convert to pandas Series and sort
genre_profit = pd.Series(genre_profit).sort_values()

# Plot
plt.figure(figsize=(12, 8))
genre_profit.plot(kind='barh', color='darkcyan')
plt.title("Average Profit by Genre", fontsize=14)
plt.xlabel("Average Profit (in $ billions)", fontsize=12)
plt.ylabel("Genre", fontsize=12)
plt.grid(axis='x', linestyle='--', alpha=0.7)
plt.show()